In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import bifacial_radiance as br

# testfolder = Path().resolve().parent.parent / 'TEMP' / 'bifacial_radiance'

testfolder = os.path.join('TEMP', 'bifacial_radiance')

# if not os.path.exists(testfolder):
#     os.makedirs(testfolder)
    
# print ("Your simulation will be stored in %s" % testfolder)

In [3]:
print(os.path.join('TEMP', 'bifacial_radiance'))

TEMP\bifacial_radiance


In [4]:
# remove unwanted results, in this case all the "fronts"
filesall = os.listdir(os.path.join('TEMP', 'bifacial_radiance','results'))
filestoclean = [e for e in filesall if e.endswith('_Front.csv')]
for cc in range(0, len(filestoclean)):
    filetoclean = filestoclean[cc]
    os.remove(os.path.join('results', filetoclean))

In [5]:
from datetime import datetime
import re
print((os.path.join(testfolder, 'results')))


TEMP\bifacial_radiance\results


In [6]:
filestarter = "irr_1axis_2023-09-16_"
filelist = sorted(os.listdir(os.path.join(testfolder, 'results')))
prefixed = [filename for filename in filelist if filename.startswith(filestarter)]
# print(prefixed)
distance = []
groundirrad = []
filenamed = []
faillist = []
Datetime = []
Ap_1 = []
Ap_2 = []
Ap_3 = []
Ap_4 = []
Ap_5 = []

print('{} files in the directory'.format(filelist.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.

for i in range (0, len(prefixed)):
    ind = prefixed[i].split('_')

    try:
        Datetime.append(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[i])[0])
        resultsDF = br.load.read1Result(os.path.join(testfolder, 'results', prefixed[i]))
        Ap_1.append(resultsDF['Wm2Back'][0])
        Ap_2.append(resultsDF['Wm2Back'][1])
        Ap_3.append(resultsDF['Wm2Back'][2])
        Ap_4.append(resultsDF['Wm2Back'][3])
        Ap_5.append(resultsDF['Wm2Back'][4])

        filenamed.append(prefixed[i])
    except:
        print(" FAILED ", i, prefixed[i])
        faillist.append(prefixed[i])
        
# resultsdf = pd.DataFrame(list(zip(distance, groundirrad)), columns = ['br_position', 'br_irradiance'])
resultsdf = pd.DataFrame(list(zip(Ap_1, Ap_2, Ap_3, Ap_4, Ap_5)), columns = ['Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5'])

resultsdf['Datetime'] = Datetime
resultsdf['Datetime'] = pd.to_datetime(resultsdf['Datetime'], format ='%Y-%m-%d_%H%M')


print(resultsdf)

147 files in the directory
47 groundscan files in the directory
         Ap_1       Ap_2       Ap_3       Ap_4       Ap_5            Datetime
0    14.66802   15.60390   16.17136   15.57604   14.63485 2023-09-16 06:00:00
1    10.65493   12.43125   12.51109   11.81129   10.44835 2023-09-16 06:15:00
2    14.30578   16.47719   16.81896   15.15688   13.87086 2023-09-16 06:30:00
3    16.40230   20.56065   20.85740   18.17289   15.89276 2023-09-16 06:45:00
4    19.27554   23.92238   22.47232   18.75018   17.53202 2023-09-16 07:00:00
5    21.11580   27.27283   23.10771   18.09979   17.00869 2023-09-16 07:15:00
6    24.34222   28.94668   23.84730   18.29130   18.22185 2023-09-16 07:30:00
7    26.95593   31.23748   24.95348   19.32661  349.30460 2023-09-16 07:45:00
8   408.58630   34.10092   26.76044   20.09225  399.53340 2023-09-16 08:00:00
9   458.84490   35.90514   27.93058   20.99389  448.57500 2023-09-16 08:15:00
10  506.18320   37.94323   29.15354   21.49717  493.91740 2023-09-16 08:30:00


In [7]:
resultsdf.to_csv('out6.csv',index=False)
print(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')))

C:\Users\cchoi2\Documents\Github\InSPIRE\Studies\BARN_Ground-Sensors\Data\BARNirrad_measured.csv


In [45]:
# Tidy the data
modeled_melt = resultsdf.melt(id_vars = ['Datetime'], var_name = 'position', value_name = 'value')
modeled_melt['datatype'] = 'modeled'

# Load the measured data
measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1)
# print(resultsdf_melt.info)
# measured.info

measured['Datetime'] = pd.to_datetime(measured['TIMESTAMP'], format ='%m/%d/%Y %H:%M')
# measured_select = measured[['Datetime', 'Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5']]
measured_melt = measured[['Datetime', 'Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5']].melt(
    id_vars = ['Datetime'],
    var_name = 'position', 
    value_name = 'value')
measured_melt['datatype'] = 'measured'

# combined = pd.merge(resultsdf_melt, measured_melt, how = 'left', on = 'Datetime')
measured_melt = pd.merge(modeled_melt[['Datetime']], measured_melt, how = 'left', on = 'Datetime')
measured_melt.drop_duplicates(inplace=True)
combined = pd.concat([modeled_melt, measured_melt])
print(combined)

# Make wider for another plot
combined_wider = combined.pivot(index=['Datetime', 'position'], columns='datatype', values = 'value').reset_index()

# Sum up the irradiance values
# combined_sum = combined.groupby(['position', 'datatype']).agg(np.sum())
combined['totalrad'] = combined.groupby(['position', 'datatype'])['value'].transform(lambda x: x.sum() / 4)
combined_summary = combined[['position','datatype','totalrad']].drop_duplicates()
# combined_sum = combined.groupby(['position', 'datatype'])['value'].transform(lambda x: x.sum() / 4)


               Datetime position     value  datatype
0   2023-09-16 06:00:00     Ap_1  14.66802   modeled
1   2023-09-16 06:15:00     Ap_1  10.65493   modeled
2   2023-09-16 06:30:00     Ap_1  14.30578   modeled
3   2023-09-16 06:45:00     Ap_1  16.40230   modeled
4   2023-09-16 07:00:00     Ap_1  19.27554   modeled
..                  ...      ...       ...       ...
690 2023-09-16 17:30:00     Ap_1  13.02133  measured
693 2023-09-16 17:30:00     Ap_2  15.19155  measured
696 2023-09-16 17:30:00     Ap_3  15.73410  measured
699 2023-09-16 17:30:00     Ap_4  15.19155  measured
702 2023-09-16 17:30:00     Ap_5  13.56388  measured

[470 rows x 4 columns]


In [46]:
print(combined_summary)

    position  datatype     totalrad
0       Ap_1   modeled  2322.563208
47      Ap_2   modeled  3512.495393
94      Ap_3   modeled  4190.799375
141     Ap_4   modeled  3841.109245
188     Ap_5   modeled  2347.201652
0       Ap_1  measured  2428.813873
3       Ap_2  measured  3824.723060
6       Ap_3  measured  4354.894320
9       Ap_4  measured  4039.359247
12      Ap_5  measured  2523.500438


In [48]:
import plotly.express as px

In [50]:
# Simple line graph of the measured vs modeled irradiance values
fig1 = px.line(combined, x = 'Datetime', y = 'value', color = 'datatype', facet_col = 'position', facet_col_wrap = 5)
fig1.show()

# 1-to-1 plot of the measured vs modeled irradiance values
fig2 = px.scatter(combined_wider, x = 'measured', y = 'modeled', color='Datetime', facet_col = 'position', facet_col_wrap=5)
fig2.show()

# side-by-side barcharts
fig3 = px.bar(combined_summary, x='position', y='totalrad', color='datatype', barmode='group')
fig3.show()